In [22]:
import pandas as pd
import numpy as np
import seaborn as sns 

In [23]:
raw_file = pd.read_csv("data/202308-divvy-tripdata.csv")
base_dataframe = pd.DataFrame(raw_file)
base_dataframe.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,ride_length,day_of_week
0,903C30C2D810A53B,electric_bike,2023-08-19 15:41,2023-08-19 15:53,LaSalle St & Illinois St,13430,Clark St & Elm St,TA1307000039,41.890721,-87.631477,41.902973,-87.63128,member,0:11:43,7
1,F2FB18A98E110A2B,electric_bike,2023-08-18 15:30,2023-08-18 15:45,Clark St & Randolph St,TA1305000030,NaN,NaN,41.884511,-87.631550,41.930000,-87.64000,member,0:15:07,6
2,D0DEC7C94E4663DA,electric_bike,2023-08-30 16:15,2023-08-30 16:27,Clark St & Randolph St,TA1305000030,NaN,NaN,41.884981,-87.630793,41.910000,-87.63000,member,0:12:29,4
3,E0DDDC5F84747ED9,electric_bike,2023-08-30 16:24,2023-08-30 16:33,Wells St & Elm St,KA1504000135,NaN,NaN,41.903105,-87.634667,41.900000,-87.62000,member,0:09:27,4
4,7797A4874BA260CA,electric_bike,2023-08-22 15:59,2023-08-22 16:20,Clark St & Randolph St,TA1305000030,NaN,NaN,41.885548,-87.632019,41.890000,-87.68000,member,0:20:54,3


In [24]:
# check for percentage of blank cells in each column of the dataframe 
base_dataframe.isna().sum()/base_dataframe.shape[0]

ride_id               0.000000
rideable_type         0.000000
started_at            0.000000
ended_at              0.000000
start_station_name    0.154101
start_station_id      0.154101
end_station_name      0.162718
end_station_id        0.162718
start_lat             0.000000
start_lng             0.000000
end_lat               0.001629
end_lng               0.001629
member_casual         0.000000
ride_length           0.000000
day_of_week           0.000000
dtype: float64

In [25]:
# removing rides with blank data
cleaned_data = base_dataframe.dropna()
# number of rows and columns of cleaned data
cleaned_data.shape

(584919, 15)

In [26]:
# dropping rows with the same start and end times
def get_matching_rows_indexes(df, col1, col2):
    """
    Get a list of indexes of rows where the values in col1 and col2 are the same.

    Parameters:
    df (pd.DataFrame): The DataFrame to check.
    col1 (str): The name of the first column.
    col2 (str): The name of the second column.

    Returns:
    list: A list of row indexes where the values in col1 and col2 are the same.
    """
    matching_indexes = df.index[df[col1] == df[col2]].tolist()
    return matching_indexes

rides_with_the_same_start_and_end_time = get_matching_rows_indexes(cleaned_data, "started_at", "ended_at")
cleaned_data = cleaned_data.drop(rides_with_the_same_start_and_end_time)


In [27]:
# getting distance of bike rides from longitude and lattitude co-ordinates
def deg2rad(deg):
    """
    Convert degrees to radians.

    Parameters:
    deg (float): Angle in degrees.

    Returns:
    float: Angle in radians.
    """
    return deg * (np.pi / 180)

def get_distance_from_lat_lon_in_km(lat1, lon1, lat2, lon2):
    """
    Calculate the distance between two points on the Earth's surface specified by latitude and longitude using the Haversine formula.

    Parameters:
    lat1 (float): Latitude of the first point in degrees.
    lon1 (float): Longitude of the first point in degrees.
    lat2 (float): Latitude of the second point in degrees.
    lon2 (float): Longitude of the second point in degrees.

    Returns:
    float: Distance between the two points in kilometers.
    """

    R = 6371  # Radius of the Earth in km
    
    dLat = deg2rad(lat2 - lat1)
    dLon = deg2rad(lon2 - lon1)
    
    a = (np.sin(dLat / 2) * np.sin(dLat / 2) +
         np.cos(deg2rad(lat1)) * np.cos(deg2rad(lat2)) *
         np.sin(dLon / 2) * np.sin(dLon / 2))
    
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    d = R * c # Distance in km
    return d

cleaned_data["ride_distance_km"] = get_distance_from_lat_lon_in_km(
    cleaned_data["start_lat"],
    cleaned_data["start_lng"],
    cleaned_data["end_lat"], 
    cleaned_data["end_lng"]    
)

In [28]:
def get_seconds(time_str):
    """
    Get seconds from hh:mm:ss formatted time.
    
    Parameters: 
    time_str (string): time in hh:mm:ss format

    Returns:
    int: time in seconds
    """
    try:
        h, m, s = list(map(int, time_str.split(":")))
        return h * 3600 + m * 60 + s
    
    except ValueError:
        return 0
    
    
def get_zero_distance_indexes(df, col):
    """
    Get a list of indexes of rows where the values in col are 0.

    Parameters:
    df (pd.DataFrame): The DataFrame to check.
    col (str): The name of the column.

    Returns:
    list: A list of row indexes where the values in col are 0
    """
    matching_indexes = df.index[df[col] == 0].tolist()
    return matching_indexes


# cleaning data of zero-length rides
rides_with_zero_time = get_zero_distance_indexes(cleaned_data, "ride_length")
cleaned_data = cleaned_data.drop(rides_with_zero_time)

# converting the hh:mm:ss format of time into seconds
cleaned_data["ride_length_seconds"] = cleaned_data["ride_length"].apply(get_seconds)

# converting member_casual to 1/0
cleaned_data["member_casual"] = cleaned_data["member_casual"].astype("category")
cleaned_data["member_casual_binary"] = cleaned_data["member_casual"].cat.codes

# exporting the cleaned data to csv format
cleaned_data.to_csv("cleaned_data/clean-202308-divvy-tripdata.csv", index=False)